# DataFrames

* a `DataFrame` is a popular data structure (in many languages) for storing data in tabular form (rows, columns)
* the Julia package `DataFrames.jl` has many functions that operate on `DataFrames` that allow us to quickly query and manipulate data
* it can handle missing values efficiently
* the Julia package `CSV.jl` allows us to quickly and flexibly read/write CSV (= comma-separated value) files from/to `DataFrame`s

You need to install these packages:
```julia
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
```
or, for pros, open a Julia [REPL](https://en.wikipedia.org/wiki/Read–eval–print_loop), hit the `]` key to go into package mode, then `add DataFrames CSV`.

* `DataFrames.jl` [documentation](http://juliadata.github.io/DataFrames.jl/stable/)
* `CSV.jl` [documentation](https://juliadata.github.io/CSV.jl/stable/)

In [1]:
# always list packages you use at the top
using DataFrames # load into namespace all of the functions, data types, variables from the `DataFrames` package
using CSV
using Printf # for print formating
using Statistics # for `mean`

## construct a `DataFrame`

In [2]:
city = ["Corvallis", "Portland", "Eugene"]
population = [57961, 647805, 168916]

df_cities = DataFrame(city=city, population=population)

# this works too of course:
#   df_cities = DataFrame(city=["Corvallis", "Portland", "Eugene"], population=[57961, 647805, 168916])

,city,population
,String,Int64
1,Corvallis,57961
2,Portland,647805
3,Eugene,168916


## add rows to a `DataFrame`

array-style (must memorize order)

In [3]:
push!(df_cities, ["Bend", 94520])
df_cities

,city,population
,String,Int64
1,Corvallis,57961
2,Portland,647805
3,Eugene,168916
4,Bend,94520


dictionary style (no need to memorize order)

In [4]:
push!(df_cities, Dict(:population => 122324, :city => "Berkeley"))
df_cities

,city,population
,String,Int64
1,Corvallis,57961
2,Portland,647805
3,Eugene,168916
4,Bend,94520
5,Berkeley,122324


## add columns to the `DataFrame`

In [5]:
# (1) one way, inserts new column as the last column
df_cities[!, :state] = vcat(["Oregon" for i = 1:4], ["California"])

# (2) another way, allows you to insert a column at specified location (e.g., 2)
insertcols!(df_cities, 2, rainfall=[51.0, 43.0, 47.0, 12.0, 25.0])
df_cities

,city,rainfall,population,state
,String,Float64,Int64,String
1,Corvallis,51.0,57961,Oregon
2,Portland,43.0,647805,Oregon
3,Eugene,47.0,168916,Oregon
4,Bend,12.0,94520,Oregon
5,Berkeley,25.0,122324,California


## how many (rows, columns)?

In [6]:
size(df_cities) # size(df_cities)[1] = # rows, size(df_cities)[2] = # cols 

(5, 4)

## what are the column names?
note that the column names are always symbols! if you have a fancy `String` you want to convert into a `Symbol`, simply e.g. `Symbol("avg salary (USD)")` will convert the `String` `"avg salary (USD)"` to a `Symbol`.

In [7]:
names(df_cities) # will be very handy when there are *many* columns

4-element Array{Symbol,1}:
 :city      
 :rainfall  
 :population
 :state     

## iterate through the rows of a `DataFrame`

In [8]:
for row in eachrow(df_cities)
    # inside the loop, we have a row (representing a data point)
    # and we can call whatever attribute of that data point we want
    @printf("population of %s, %s is %d\n", row[:city], row[:state], row[:population])
end

population of Corvallis, Oregon is 57961
population of Portland, Oregon is 647805
population of Eugene, Oregon is 168916
population of Bend, Oregon is 94520
population of Berkeley, California is 122324


## retreive a column
the columns are `Array`s!

In [9]:
# (1) treating the DataFrame like a multi-dimensional array
df_cities[!, :population] # ! = all the rows

5-element Array{Int64,1}:
  57961
 647805
 168916
  94520
 122324

In [10]:
# (2) treating the DataFrame like a `Struct`
df_cities.population

5-element Array{Int64,1}:
  57961
 647805
 168916
  94520
 122324

## retreive a row

In [11]:
df_cities[2, :]

,city,rainfall,population,state
,String,Float64,Int64,String
2,Portland,43.0,647805,Oregon


## retreive an entry
that is, retreive an attribute of a data point (= row)

In [12]:
df_cities[2, :population] # population of Portland

647805

## retreive a combination of rows/columns

like an array, we can retreive, say, the first 3 rows and only the `:population` and `:city` columns

In [13]:
df_cities[1:3, [:population, :city]]

,population,city
,Int64,String
1,57961,Corvallis
2,647805,Portland
3,168916,Eugene


so dataframes behave like arrays except their columns are labeled and queried by a `Symbol`

## querying a dataframe

say we want all the rows where the city is listed as "Corvallis"

(1) query a `DataFrame` much like array slicing, via an array of bits

In [14]:
df_cities[df_cities[!, :city] .== "Corvallis", :]

,city,rainfall,population,state
,String,Float64,Int64,String
1,Corvallis,51.0,57961,Oregon


(2) the `filter` function.
* the first argument is a function that operates on a row of the `DataFrame` (treating the row as a `Dict`) and returns `true` if we want to keep that row and `false` if we want to throw it away
* the second argument is the dataframe
* there is also a `filter!` function that will remove these rows from the dataframe instead of returning a copy with the relevant rows removed

In [15]:
filter(row -> row[:city] == "Corvallis", df_cities)

,city,rainfall,population,state
,String,Float64,Int64,String
1,Corvallis,51.0,57961,Oregon


... more examples:

* all the rows where the population is less than 500,000

In [16]:
df_cities[df_cities[!, :population] .< 500000, :]

,city,rainfall,population,state
,String,Float64,Int64,String
1,Corvallis,51.0,57961,Oregon
2,Eugene,47.0,168916,Oregon
3,Bend,12.0,94520,Oregon
4,Berkeley,25.0,122324,California


... all the rows where cities are in the state of Oregon

In [17]:
filter(row -> row[:state] == "Oregon", df_cities)

,city,rainfall,population,state
,String,Float64,Int64,String
1,Corvallis,51.0,57961,Oregon
2,Portland,43.0,647805,Oregon
3,Eugene,47.0,168916,Oregon
4,Bend,12.0,94520,Oregon


## sorting a dataframe

e.g. permute rows so that cities are listed by `:population` in reverse (`rev`) order

In [18]:
sort!(df_cities, :population, rev=true) # ! means the input dataframe was modified. now the rows are re-ordered
df_cities

,city,rainfall,population,state
,String,Float64,Int64,String
1,Portland,43.0,647805,Oregon
2,Eugene,47.0,168916,Oregon
3,Berkeley,25.0,122324,California
4,Bend,12.0,94520,Oregon
5,Corvallis,51.0,57961,Oregon


## `groupby`: split the rows of a `DataFrame` into groups

the `groupby` command (common in many languages) partitions the rows in the `DataFrame` into multiple `DataFrame`s such that the rows in each `DataFrame` share a common attribute. this is very useful for them performing computations on each group separately.

e.g., group the rows in `df_cities` (that each represent a city) by state.

In [19]:
gb_state = groupby(df_cities, :state)

,city,rainfall,population,state
,String,Float64,Int64,String
1,Portland,43.0,647805,Oregon
2,Eugene,47.0,168916,Oregon
3,Bend,12.0,94520,Oregon
4,Corvallis,51.0,57961,Oregon
,city,rainfall,population,state
,String,Float64,Int64,String
1,Berkeley,25.0,122324,California


In [20]:
for df_by_state in gb_state
    # to which state does this group correspond?
    this_state = df_by_state[1, :state]
    
    # operate on chunks of data in this state
    #  `df_by_state` DataFrame has data *only* from `this_state
    
    # e.g. how many cities are in this state?
    nb_cities_in_this_state = size(df_by_state)[1]
    
    # e.g. what is the average rainfall in this state?
    avg_rainfall_in_this_state = mean(df_by_state[:, :rainfall])
    
    # * is string concatenation
    @printf("%s has %d cities with avg. rainfaill %.2f\n",
        this_state, nb_cities_in_this_state, avg_rainfall_in_this_state)
end

Oregon has 4 cities with avg. rainfaill 38.25
California has 1 cities with avg. rainfaill 25.00


## `by`: split into groups, apply a function, combine back

`by(df, :key, :col => f)` will split the `DataFrame` `df` into groups in the `:key` column, then apply function `f` to the `:col` column.

e.g., group by state, take mean of the rainfall column. see [docs](https://juliadata.github.io/DataFrames.jl/stable/lib/functions/#DataFrames.by).

In [21]:
df_mean_rainfall = by(df_cities, :state, :rainfall => mean)

,state,rainfall_mean
,String,Float64
1,Oregon,38.25
2,California,25.0


you can also represent the function to be applied as an anonymous function.

In [22]:
df_mean_rainfall = by(df_cities, :state, :rainfall => col -> mean(col))

,state,rainfall_function
,String,Float64
1,Oregon,38.25
2,California,25.0


`aggregate` is similar to `by` but it applies a function to all columns in the grouped `DataFrames`. we illustrate it here with only the `:rainfall` and `:population` columns since it wouldn't make sense to take the mean of the `:city` column of `String`s. see [docs](https://juliadata.github.io/DataFrames.jl/stable/lib/functions/#DataFrames.aggregate)

In [23]:
aggregate(df_cities[:, [:state, :rainfall, :population]], :state, mean)

,state,rainfall_mean,population_mean
,String,Float64,Float64
1,Oregon,38.25,2.42301e5
2,California,25.0,122324.0


## renaming a column

In [24]:
rename!(df_mean_rainfall, :rainfall_function => :avg_rainfall)
df_mean_rainfall

,state,avg_rainfall
,String,Float64
1,Oregon,38.25
2,California,25.0


## read a CSV from a file

CSV = comma-separated values, a very common format for storing tables of data

In [25]:
run(`cat incomes.csv`) # running shell command from Julia! this is what the file looks like

city,per_capita_income
Corvallis,19317
Berkeley,38896
San Franscisco,52261


Process(`cat incomes.csv`, ProcessExited(0))

In [26]:
df_incomes = CSV.read("incomes.csv")

,city,per_capita_income
,String,Int64
1,Corvallis,19317
2,Berkeley,38896
3,San Franscisco,52261


## joins
combine two dataframes that have different columns

there are [seven types of joins](http://juliadata.github.io/DataFrames.jl/stable/man/joins/). let's illustrate two here.

goal: join information about *cities* from `df_cities` and `df_incomes`. thus the *key* here is `:city` since we aim to combine rows of the two `DataFrames` on the basis of the `:city` column.

subtlety here: 
* San Francisco is in `df_incomes` but missing from `df_cities`
* Bend, Eugene, Portland are in `df_cities` but missing from `df_incomes`

### inner join
only keep rows where the city is common between the two `DataFrames`
(throw out rows that aren't common between the two)

In [27]:
df = join(df_cities, df_incomes, on=:city, kind=:inner)

,city,rainfall,population,state,per_capita_income
,String,Float64,Int64,String,Int64
1,Berkeley,25.0,122324,California,38896
2,Corvallis,51.0,57961,Oregon,19317


### outer join
keep all rows, fill with `missing` values when an attribute is missing in either `DataFrame`
(keep rows that aren't common between the two)

In [28]:
df = join(df_cities, df_incomes, on=:city, kind=:outer)

,city,rainfall,population,state,per_capita_income
,String,Float64⍰,Int64⍰,String⍰,Int64⍰
1,Portland,43.0,647805,Oregon,missing
2,Eugene,47.0,168916,Oregon,missing
3,Berkeley,25.0,122324,California,38896
4,Bend,12.0,94520,Oregon,missing
5,Corvallis,51.0,57961,Oregon,19317
6,San Franscisco,missing,missing,missing,52261


## missing values
Julia has a data type to efficiently handle missing values

In [29]:
typeof(missing)

Missing

columns with missing values are arrays of whatever type but `Union`'d with the `Missing` type

In [30]:
df[!, :population]

6-element Array{Union{Missing, Int64},1}:
 647805       
 168916       
 122324       
  94520       
  57961       
       missing

remove all rows that have a missing attribute

In [31]:
dropmissing(df) # returns a copy. 
# dropmissing!(df) would remove these rows with missingfrom the dataframe passed to it.

,city,rainfall,population,state,per_capita_income
,String,Float64,Int64,String,Int64
1,Berkeley,25.0,122324,California,38896
2,Corvallis,51.0,57961,Oregon,19317


remove all rows with a missing per capita income

In [32]:
dropmissing(df, :per_capita_income)

,city,rainfall,population,state,per_capita_income
,String,Float64⍰,Int64⍰,String⍰,Int64
1,Berkeley,25.0,122324,California,38896
2,Corvallis,51.0,57961,Oregon,19317
3,San Franscisco,missing,missing,missing,52261


## delete rows
let's delete the row corresponding to Eugene

In [33]:
deleterows!(df_cities, 2) # Eugene was the second row
df_cities

,city,rainfall,population,state
,String,Float64,Int64,String
1,Portland,43.0,647805,Oregon
2,Berkeley,25.0,122324,California
3,Bend,12.0,94520,Oregon
4,Corvallis,51.0,57961,Oregon


## delete columns

we can delete columns, on the other hand, by specifying the name of the column as a `Symbol`.

In [34]:
deletecols(df_cities, :state)

┌ Warning: `deletecols(df::DataFrame, inds; copycols::Bool=true)` is deprecated, use `select(df, Not(inds), copycols=copycols)` instead.
│   caller = top-level scope at In[34]:1
└ @ Core In[34]:1


,city,rainfall,population
,String,Float64,Int64
1,Portland,43.0,647805
2,Berkeley,25.0,122324
3,Bend,12.0,94520
4,Corvallis,51.0,57961


## find unique entries of a column
(`unique` is a function in Julia `Base`)

In [35]:
unique(df[:, :state])

3-element Array{Union{Missing, String},1}:
 "Oregon"    
 "California"
 missing     

## delete rows that are not unique
i.e. delete duplicate rows. first, let's make a duplicate row.

In [36]:
push!(df_cities, df_cities[1, :]) # copy first row
df_cities

,city,rainfall,population,state
,String,Float64,Int64,String
1,Portland,43.0,647805,Oregon
2,Berkeley,25.0,122324,California
3,Bend,12.0,94520,Oregon
4,Corvallis,51.0,57961,Oregon
5,Portland,43.0,647805,Oregon


now pass the `DataFrame` to the `unique!` function, which will remove the duplicate rows. remember, the `!` means remove these duplicate rows from the passed dataframe as opposed to returning a copy of the `DataFrame` with the duplicate rows removed. the latter can be achieved by `unique(df_cities)`, i.e. by omitting the `!`.

In [37]:
unique!(df_cities)
df_cities

,city,rainfall,population,state
,String,Float64,Int64,String
1,Portland,43.0,647805,Oregon
2,Berkeley,25.0,122324,California
3,Bend,12.0,94520,Oregon
4,Corvallis,51.0,57961,Oregon


## write `DataFrame` to CSV

In [38]:
CSV.write("joined_df.csv", df)

run(`cat joined_df.csv`)

city,rainfall,population,state,per_capita_income
Portland,43,647805,Oregon,
Eugene,47,168916,Oregon,
Berkeley,25,122324,California,38896
Bend,12,94520,Oregon,
Corvallis,51,57961,Oregon,19317
San Franscisco,,,,52261


Process(`cat joined_df.csv`, ProcessExited(0))